## Adding a Long-Only Strategy

Let´s implement a Long-only Price & Volume Strategy with the following Parameters:

- bar_length = __1m__ 
- Return Threshold: __0%__
- Low and High Volume Change Threshold: __[-3.0, 3.0]__
- effectively: very simple Contrarian Strategy, sell (buy) if most recent 1m return was positive (negative)
- not profitable but many position changes & trades (good for demonstration purpos

bar_length = 1m: 시간 단위로 데이터를 처리하며, 각 바(bar)의 길이는 1분. 이는 거래 데이터를 1분 단위로 분석하고, 각 분마다의 수익률을 계산하는 것을 의미한다.

Return Threshold: 0%: 수익률 임계값은 0%로 설정되어있다. 이는 수익률이 0%를 초과할 경우 매수하거나, 수익률이 0% 미만일 경우 매도하는 전략을 취할 것을 의미한다.

Low and High Volume Change Threshold: [-3.0, 3.0]: 거래량 변화 임계값은 -3.0과 3.0 사이이다. 이는 거래량의 변화가 -3.0%에서 3.0% 사이에 있을 때 거래를 수행할 것을 의미한다. 이것은 거래량의 변화를 고려하여 거래를 수행하는 것을 나타낸다.

effectively: very simple Contrarian Strategy, sell (buy) if most recent 1m return was positive (negative): 이것은 매우 간단한 반대 전략을 나타낸다. 최근 1분 동안의 수익률이 양수(음수)이면 매도(매수)할 것이다. 즉, 시장의 추세에 반대하는 전략을 취하고자한다.

not profitable but many position changes & trades (good for demonstration purposes): 이 전략은 수익성이 없을 수 있지만, 많은 포지션 변경과 거래가 이루어진다. 이는 데모 목적으로 좋은 것으로, 전략의 작동 방식을 시연하기에 적합하다는 것을 의미한다.


In [1]:
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
api_key = "TGV2OZgIKqiCEWvhVUAUDGBB99Eixo1z2ILlZ0h3lRdutfklTwRlqzPYpIazsYcn"
secret_key = "CrgLtKo6MRxdL6tJRNKokKPKOAWa0Rn59Q9KfX57yal0E1WTqz79jk2IT03g6VcZ"
#test api_key
test_api_Key = "ni1RGarrqKhXFMnmqZt2EEbEJkpGDXs4AxHml3r7aevzg7uqUR1rTFbNngWiTOln"
test_secret_Key = "HqFm7zQNQPOEJdYMgLho7engJbah6FAH98Eqf607h467CB9RM9e6dHDytnQbKGgk"

In [3]:
client = Client(api_key = test_api_Key, api_secret = test_secret_Key, tld = "com", testnet = True)

In [4]:
class LongOnlyTrader():
    
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh):
        
        self.symbol = symbol
        self.bar_length = bar_length
        self.available_intervals = ["1m", "3m", "5m", "15m", "30m", "1h", "2h", "4h", "6h", "8h", "12h", "1d", "3d", "1w", "1M"]
        
        #*****************add strategy-specific attributes here******************
        self.return_thresh = return_thresh
        self.volume_thresh = volume_thresh
        #************************************************************************
    
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length,
                                 days = historical_days)
            #callback함수 self.stream_candles에 굳이 msg 인자값을 안넣어도 되는 이유는 start_kline_socket
            #함수가 콜백함수에 파라미터를 전달하기 떄문.
            self.twm.start_kline_socket(callback = self.stream_candles,
                                        symbol = self.symbol, interval = self.bar_length)
        # "else" to be added later in the course 
    
    def get_most_recent(self, symbol, interval, days):
        print("get_most_recent()메서드 실행....bar_length..?"+str(interval))
        now = datetime.utcnow()
        past = str(now - timedelta(days = days))
    
        bars = client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)
        df = pd.DataFrame(bars)
        df["Date"] = pd.to_datetime(df.iloc[:,0], unit = "ms")
        df.columns = ["Open Time", "Open", "High", "Low", "Close", "Volume",
                      "Clos Time", "Quote Asset Volume", "Number of Trades",
                      "Taker Buy Base Asset Volume", "Taker Buy Quote Asset Volume", "Ignore", "Date"]
        df = df[["Date", "Open", "High", "Low", "Close", "Volume"]].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = "coerce")
        df["Complete"] = [True for row in range(len(df)-1)] + [False]
        
        self.data = df
    
    def stream_candles(self, msg):
        print("stream_candles()메서드 실행")
        # extract the required items from msg
        event_time = pd.to_datetime(msg["E"], unit = "ms")
        start_time = pd.to_datetime(msg["k"]["t"], unit = "ms")
        first   = float(msg["k"]["o"])
        high    = float(msg["k"]["h"])
        low     = float(msg["k"]["l"])
        close   = float(msg["k"]["c"])
        volume  = float(msg["k"]["v"])
        complete=       msg["k"]["x"]
    
        # print out
        print(".", end = "", flush = True) # just print something to get a feedback (everything OK) 
    
        # feed df (add new bar / update latest bar)
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare features and define strategy/trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
        
    def define_strategy(self): # "strategy-specific"
        
        df = self.data.copy()
        
        #******************** define your strategy here ************************
        df = df[["Close", "Volume"]].copy()
        df["returns"] = np.log(df.Close / df.Close.shift())
        df["vol_ch"] = np.log(df.Volume.div(df.Volume.shift(1)))
        df.loc[df.vol_ch > 3, "vol_ch"] = np.nan
        df.loc[df.vol_ch < -3, "vol_ch"] = np.nan  
        
        cond1 = df.returns >= self.return_thresh
        cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        
        df["position"] = 1
        df.loc[cond1 & cond2, "position"] = 0
        #***********************************************************************
        
        self.prepared_data = df.copy()
        

In [5]:
symbol = "BTCUSDT"
bar_length = "1m"
return_thresh = 0
volume_thresh = [-3, 3]

In [6]:
trader = LongOnlyTrader(symbol = symbol, bar_length = bar_length,
                        return_thresh = return_thresh, volume_thresh = volume_thresh)

In [7]:
trader.start_trading(historical_days = 1/24)

get_most_recent()메서드 실행....bar_length..?1m
stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실행
.stream_candles()메서드 실

In [8]:
trader.twm.stop()

stream_candles()메서드 실행
.

In [9]:
trader.data

,Open,High,Low,Close,Volume,Complete
Date,,,,,,
2024-03-27 05:40:00,70456.29,70479.50,70441.10,70441.10,0.10680,True
2024-03-27 05:41:00,70441.11,70479.06,70441.09,70479.06,0.14315,True
2024-03-27 05:42:00,70479.10,70481.63,70444.00,70458.00,0.10482,True
2024-03-27 05:43:00,70458.00,70469.99,70444.75,70469.98,0.08680,True
2024-03-27 05:44:00,70469.99,70479.61,70443.00,70448.18,0.27623,True
...,...,...,...,...,...,...
2024-03-27 06:40:00,70284.75,70284.76,70243.76,70243.76,19.42543,True
2024-03-27 06:41:00,70243.77,70244.00,70210.26,70210.26,9.99661,True
2024-03-27 06:42:00,70210.26,70247.64,70210.26,70233.09,11.09180,True


In [11]:
import pytz
utc_now = datetime.now(pytz.utc)
print("UTC 현재 시간:", utc_now)

UTC 현재 시간: 2024-03-27 06:45:51.823886+00:00


In [13]:
trader.prepared_data
#거래표지션이 0일때 팔아야한다.

,Close,Volume,returns,vol_ch,position
Date,,,,,
2024-03-27 05:40:00,70441.10,0.10680,NaN,NaN,1
2024-03-27 05:41:00,70479.06,0.14315,0.000539,0.292935,0
2024-03-27 05:42:00,70458.00,0.10482,-0.000299,-0.311648,1
2024-03-27 05:43:00,70469.98,0.08680,0.000170,-0.188638,0
2024-03-27 05:44:00,70448.18,0.27623,-0.000309,1.157627,1
...,...,...,...,...,...
2024-03-27 06:39:00,70284.76,5.51408,-0.000644,NaN,1
2024-03-27 06:40:00,70243.76,19.42543,-0.000584,1.259278,1
2024-03-27 06:41:00,70210.26,9.99661,-0.000477,-0.664337,1
